# Multi-armed Bandits

![mab](imgs/mab.jpg)


O **Multi-armed bandits (MaB)** é um problema específico de aprendizado por reforço na qual temos **k** diferentes opções (ou ações) $A_1, A_2, \cdots A_k$, cada uma associada a uma distribuição real $R = \{R_1, R_2, \cdots, R_k\}$ correspondente às **recompensas**, onde $\mu_1, \mu_2, \cdots, \mu_k$ são as médias dessas distribuições respectivamente.


No MaB temos um agente que não conhece *a priori* as distribuições de recompensa. Para descobrir ele executa iterativamente uma ação $A_t$ de acordo com a sua **política** e recebe uma recompensa $r_t$ (amostrada de $R_t$). O objetivo do agente é encontrar a ação que contém a maior recompensa esperada, ou seja, ele precisa obter uma boa estimativa das recompensas de cada ação, essa estimativa chamamos de valor **Q($A_i$)** da ação. O algoritmo base para o MaB é:

```
1. Obtém uma estimativa inicial de recompensa de cada uma das k ações
2. Para cada episódio t até T:
3.     Seleciona uma ação de acordo com uma política
4.     Obtém a recompensa dessa ação
5.     Atualiza a estimativa da recompensa da ação escolhida
```

In [1]:
from bernoulli_trial import BernoulliTrial
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import os
import re
from scipy import stats

# Arms

In [2]:
probabilities = [0.2, 0.5, 0.9, 0.35, 0.2]

n_arms = len(probabilities)
arms = [BernoulliTrial(p) for p in probabilities]
print(f'Best arm is {np.argmax(probabilities)}')

Best arm is 2


In [3]:
NUM_SIMULATIONS = 10
HORIZON = 500

# Epsilon Greedy

O agente com uma probabilidade $\epsilon$ executa uma ação aleatória e com uma probabilidade $1 - \epsilon$ executa a ação com a melhor estimativa (em caso de empates, uma é escolhida aleatoriamente).

$$a = \begin{cases}\text{aleatória}, \text{   com probabilidade $\epsilon$} \\
                    max(Q(A_i)), \text{    com probabilidade $1-\epsilon$}\end{cases}$$

In [4]:
from algorithms.epsilon_greedy import EpsilonGreedy

In [5]:
epsilon_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
columns=['epsilon', 'sim_num', 'episode', 'chosen_arm'] + [f'arm_{arm}' for arm in range(n_arms)]
df_epsilon = pd.DataFrame(columns=columns, dtype=float)



for epsilon in epsilon_range:
    for sim_num in range(NUM_SIMULATIONS):
        epsilon_greedy = EpsilonGreedy(epsilon, n_arms)
        for episode in range(HORIZON):
            chosen_arm = epsilon_greedy.select_arm()
            reward = arms[chosen_arm].draw()
            epsilon_greedy.update(chosen_arm, reward)
            
            new_row = {'epsilon': epsilon, 'sim_num': sim_num, 'episode': episode, 'chosen_arm': chosen_arm}
            new_row.update({f'arm_{arm}': value for arm, value in enumerate(epsilon_greedy.values)})
            
            df_epsilon = df_epsilon.append(new_row, ignore_index=True)

df_epsilon

,epsilon,sim_num,episode,chosen_arm,arm_0,arm_1,arm_2,arm_3,arm_4
0,0.1,0.0,0.0,2.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.1,0.0,1.0,1.0,0.000000,1.000000,0.000000,0.000000,0.000000
2,0.1,0.0,2.0,1.0,0.000000,1.000000,0.000000,0.000000,0.000000
3,0.1,0.0,3.0,1.0,0.000000,1.000000,0.000000,0.000000,0.000000
4,0.1,0.0,4.0,1.0,0.000000,0.750000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
44995,0.9,9.0,495.0,4.0,0.261364,0.515152,0.900709,0.283951,0.206897
44996,0.9,9.0,496.0,4.0,0.261364,0.515152,0.900709,0.283951,0.204545
44997,0.9,9.0,497.0,2.0,0.261364,0.515152,0.901408,0.283951,0.204545
44998,0.9,9.0,498.0,2.0,0.261364,0.515152,0.902098,0.283951,0.204545


In [18]:
aux = df_epsilon[df_epsilon['epsilon'] == 0.3].groupby('episode').mean().reset_index()
aux

,episode,epsilon,sim_num,chosen_arm,arm_0,arm_1,arm_2,arm_3,arm_4
0,0.0,0.3,4.5,2.3,0.100000,0.000000,0.300000,0.000000,0.000000
1,1.0,0.3,4.5,2.1,0.100000,0.100000,0.400000,0.000000,0.150000
2,2.0,0.3,4.5,2.4,0.100000,0.100000,0.550000,0.200000,0.100000
3,3.0,0.3,4.5,2.2,0.100000,0.100000,0.666667,0.150000,0.116667
4,4.0,0.3,4.5,2.4,0.100000,0.100000,0.675000,0.150000,0.083333
...,...,...,...,...,...,...,...,...,...
495,495.0,0.3,4.5,2.2,0.239536,0.496853,0.898848,0.368189,0.186590
496,496.0,0.3,4.5,1.8,0.238961,0.496853,0.898808,0.368189,0.186590
497,497.0,0.3,4.5,1.7,0.236671,0.496853,0.899007,0.366711,0.186590
498,498.0,0.3,4.5,1.7,0.236671,0.498654,0.898890,0.365441,0.186590


In [42]:
for episode in range(HORIZON):
    plt.figure(figsize=(7,7))
    x = range(n_arms)
    y = df_epsilon.loc[episode, [f'arm_{arm}' for arm in range(n_arms)]].values
    plt.scatter(x, y, marker='o', s=100)
    plt.xlabel('Arm')
    plt.ylabel('Estimated reward')
    plt.xticks(np.arange(0, 5, 1))
    plt.yticks(np.arange(0, 1, 0.1))
    plt.title('Estimated reward for each arm')
    plt.savefig(f'imgs/epsilon_greedy/episode_{episode}')
    plt.close()

In [44]:
images = []
folder = 'imgs/epsilon_greedy'
for episode in range(HORIZON):
    filename = f'{folder}/episode_{episode}.png'
    images.append(imageio.imread(filename))
imageio.mimsave(f'gifs/epsilon_greedy.gif', images)

![epsilon_greedy](gifs/epsilon_greedy.gif)

# Upper Confidence Bound (UCB)

Para cada arm, temos a estimativa da sua recompensa, que é calculada da mesma maneira que no epsilon greedy. Porém no UCB também mantemos um limite de confiança superior, que diz qual a confiança que temos sober a nossa estimativa. O UCB é calculado da seguinte maneira:
    $$\sqrt{\dfrac{2*\ln(N)}{N_i}}$$

In [10]:
from algorithms.ucb import UCB

In [11]:
columns=['sim_num', 'episode', 'chosen_arm'] + [f'arm_{arm}' for arm in range(n_arms)] + [f'ucb_{arm}' for arm in range(n_arms)]
df_ucb = pd.DataFrame(columns=columns, dtype=float)

for sim_num in range(NUM_SIMULATIONS):
    ucb = UCB(n_arms)
    for episode in range(HORIZON):
        chosen_arm = ucb.select_arm()
        reward = arms[chosen_arm].draw()
        ucb.update(chosen_arm, reward)
        
        values, ucbs = ucb.ucb_values()
        new_row = {'sim_num': sim_num, 'episode': episode, 'chosen_arm': chosen_arm}
        new_row.update({f'arm_{arm}': value for arm, value in enumerate(values)})
        new_row.update({f'ucb_{arm}': ucb for arm, ucb in enumerate(ucbs)})
        df_ucb = df_ucb.append(new_row, ignore_index=True)
        

df_ucb

,sim_num,episode,chosen_arm,arm_0,arm_1,arm_2,arm_3,arm_4,ucb_0,ucb_1,ucb_2,ucb_3,ucb_4
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.0,1.0,0.000000,1.0,0.000000,0.000000,0.000000,1.176822,2.176822,37.232974,37.232974,37.232974
2,0.0,2.0,2.0,0.000000,1.0,1.000000,0.000000,0.000000,1.481563,2.481563,2.481563,46.874562,46.874562
3,0.0,3.0,3.0,0.000000,1.0,1.000000,0.000000,0.000000,1.664277,2.664277,2.664277,1.664277,52.655377
4,0.0,4.0,4.0,0.000000,1.0,1.000000,0.000000,0.000000,1.793226,2.793226,2.793226,1.793226,1.793226
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,9.0,495.0,2.0,0.222222,0.5,0.882051,0.392857,0.222222,1.052633,1.043640,1.060457,1.058673,1.052633
4996,9.0,496.0,2.0,0.222222,0.5,0.882353,0.392857,0.222222,1.052767,1.043728,1.060559,1.058781,1.052767
4997,9.0,497.0,2.0,0.222222,0.5,0.882653,0.392857,0.222222,1.052902,1.043816,1.060661,1.058889,1.052902
4998,9.0,498.0,2.0,0.222222,0.5,0.882952,0.392857,0.222222,1.053036,1.043904,1.060761,1.058997,1.053036


In [30]:
for episode in range(HORIZON):
    plt.figure(figsize=(7,7))
    x = range(n_arms)
    y = df_ucb.loc[episode, [f'arm_{arm}' for arm in range(n_arms)]].values
    ucbs = df_ucb.loc[episode, [f'ucb_{arm}' for arm in range(n_arms)]].values
    ucb_bar = np.array([[0, ucb] for ucb in ucbs]).transpose()
    plt.errorbar(x, y, yerr=ucb_bar, fmt='o')
    plt.xlabel('Arm')
    plt.ylabel('Estimated reward')
    plt.xticks(np.arange(0, 5, 1))
    plt.yticks(np.arange(0, 5, 1))
    plt.title('Estimated reward for each arm')
    plt.savefig(f'imgs/ucb/episode_{episode}')
    plt.close()

In [31]:
images = []
folder = 'imgs/ucb'
for episode in range(HORIZON):
    filename = f'{folder}/episode_{episode}.png'
    images.append(imageio.imread(filename))
imageio.mimsave(f'gifs/ucb.gif', images)

![ucb](gifs/ucb.gif)

# Thompson Sampling

In [4]:
from algorithms.thompson_sampling import ThompsonSampling
import random

In [5]:
columns=['sim_num', 'episode', 'chosen_arm'] + [f'alpha_{arm}' for arm in range(n_arms)] + [f'beta_{arm}' for arm in range(n_arms)]
df_thompson = pd.DataFrame(columns=columns, dtype=float)


for sim_num in range(NUM_SIMULATIONS):
    thompson_sampling = ThompsonSampling(n_arms)
    for episode in range(HORIZON):
        rhos = thompson_sampling.select_arm()
        chosen_arm = random.choice([i for i, v in enumerate(rhos) if v == max(rhos)])
        reward = arms[chosen_arm].draw()
        thompson_sampling.update(chosen_arm, reward)
        
        
        new_row = {'sim_num': sim_num, 'episode': episode, 'chosen_arm': chosen_arm}
        new_row.update({f'alpha_{arm}': alpha for arm, alpha in enumerate(thompson_sampling.alpha)})
        new_row.update({f'beta_{arm}': beta for arm, beta in enumerate(thompson_sampling.beta)})
        df_thompson = df_thompson.append(new_row, ignore_index=True)

df_thompson

,sim_num,episode,chosen_arm,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,beta_0,beta_1,beta_2,beta_3,beta_4
0,0.0,0.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,0.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,0.0,2.0,0.0,2.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0
3,0.0,3.0,3.0,2.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0
4,0.0,4.0,2.0,2.0,2.0,4.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,9.0,495.0,2.0,2.0,11.0,436.0,2.0,2.0,4.0,5.0,46.0,4.0,4.0
4996,9.0,496.0,2.0,2.0,11.0,437.0,2.0,2.0,4.0,5.0,46.0,4.0,4.0
4997,9.0,497.0,2.0,2.0,11.0,438.0,2.0,2.0,4.0,5.0,46.0,4.0,4.0
4998,9.0,498.0,2.0,2.0,11.0,439.0,2.0,2.0,4.0,5.0,46.0,4.0,4.0


In [6]:
for episode in range(HORIZON):
    plt.figure(figsize=(10, 7))
    for arm in range(n_arms):
        x = np.arange (0, 1.001, 0.001)
        alpha = df_thompson.loc[episode, f'alpha_{arm}']
        beta = df_thompson.loc[episode, f'beta_{arm}']
        y = stats.beta.pdf(x, alpha, beta)
        plt.plot(x, y, label=f'arm {arm}')
    plt.legend(loc='upper left')
    plt.xlabel('Expected Reward')
    plt.ylabel('Probability Density')
    plt.yticks([])
    plt.title('Probability Distribution of Each Arm')
    plt.savefig(f'imgs/thompson/episode_{episode}')
    plt.close()

In [7]:
images = []
folder = 'imgs/thompson'
for episode in range(HORIZON):
    filename = f'{folder}/episode_{episode}.png'
    images.append(imageio.imread(filename))
imageio.mimsave(f'gifs/thompson.gif', images)

![thompson](gifs/thompson.gif)